In [1]:
import os
import osgeo
import pynldas2 as nldas

from shapely import Polygon, Point

In [2]:
import importlib
importlib.reload(nldas);

In [3]:
GEOM = Polygon(
    [[-69.77, 45.07], [-69.31, 45.07], [-69.31, 45.45], [-69.77, 45.45], [-69.77, 45.07]]
)
VAR = ["prcp", "pet"]
DEF_CRS = 4326
ALT_CRS = 3542
COORDS = (-1431147.7928, 318483.4618)
START = "2022-01-01"
END = "2022-01-31"
CONN = 1 if int(os.environ.get("GH_CI", 0)) else 4

In [4]:
nldas

<module 'pynldas2' (<_frozen_importlib_external.NamespaceLoader object at 0x000001BBB327AD90>)>

In [5]:
clm = nldas.get_bycoords(COORDS, START, END, crs=ALT_CRS, variables=['prcp','pet'], n_conn=CONN, source="grib")

AttributeError: module 'pynldas2' has no attribute 'get_bycoords'

In [6]:
clm

NameError: name 'clm' is not defined

In [25]:
clm.pet.mean()

0.12424919346774194

In [26]:
clm.prcp.mean()

0.005780376344086022

In [9]:
clm = nldas.get_bycoords(
        COORDS, START, END, crs=ALT_CRS, source=SOURCE_NETCDF, variables='prcp', n_conn=CONN
    )

In [16]:
clm = nldas.get_bycoords([COORDS], START, END, crs=ALT_CRS, variables=VAR, n_conn=CONN, source='netcdf')

In [17]:
clm

,pet,prcp
time,,


In [8]:
from pynldas2.exceptions import InputRangeError, InputTypeError, InputValueError, NLDASServiceError
import pandas as pd

NLDAS_VARS_GRIB = {
    "prcp": {"nldas_name": "APCPsfc", "long_name": "Precipitation hourly total", "units": "mm"},
    "pet": {"nldas_name": "PEVAPsfc", "long_name": "Potential evaporation", "units": "mm"},
    "temp": {"nldas_name": "TMP2m", "long_name": "2-m above ground temperature", "units": "K"},
    "wind_u": {
        "nldas_name": "UGRD10m",
        "long_name": "10-m above ground zonal wind",
        "units": "m/s",
    },
    "wind_v": {
        "nldas_name": "VGRD10m",
        "long_name": "10-m above ground meridional wind",
        "units": "m/s",
    },
    "rlds": {
        "nldas_name": "DLWRFsfc",
        "long_name": "Surface DW longwave radiation flux",
        "units": "W/m^2",
    },
    "rsds": {
        "nldas_name": "DSWRFsfc",
        "long_name": "Surface DW shortwave radiation flux",
        "units": "W/m^2",
    },
    "humidity": {
        "nldas_name": "SPFH2m",
        "long_name": "2-m above ground specific humidity",
        "units": "kg/kg",
    },
}

NLDAS_VARS_NETCDF = {
    "prcp": {"nldas_name": "Rainf", "long_name": "Total precipitation", "units": "kg/m^2"},
    "rlds": {
        "nldas_name": "LWdown",
        "long_name": "Surface downward longwave radiation",
        "units": "W/m^2",
    },
    "rsds": {
        "nldas_name": "SWdown",
        "long_name": "Surface downward shortwave radiation",
        "units": "W/m^2",
    },
    "pet": {"nldas_name": "PotEvap", "long_name": "Potential evaporation", "units": "kg/m^2"},
    "psurf": {"nldas_name": "PSurf", "long_name": "Surface pressure", "units": "Pa"},
    "humidity": {
        "nldas_name": "Qair",
        "long_name": "2-m above ground specific humidity",
        "units": "kg/kg",
    },
    "temp": {
        "nldas_name": "Tair",
        "long_name": "2-m above ground temperature",
        "units": "K",
    },
    "wind_u": {
        "nldas_name": "Wind_E",
        "long_name": "U wind component at 10-m above the surface",
        "units": "m/s",
    },
    "wind_v": {
        "nldas_name": "Wind_N",
        "long_name": "V wind component at 10-m above the surface",
        "units": "m/s",
    },
}


def check_inputs(
    start_date: str,
    end_date: str,
    variables: str | list[str] | None = None,
    snow: bool = False,
    source: str = "grib",
) -> tuple[list[pd.Timestamp], list[str], dict[str, dict[str, str]]]:
    """Check inputs."""
    start = pd.to_datetime(start_date)
    end = pd.to_datetime(end_date) + pd.Timedelta("1D")
    if start < pd.to_datetime("1979-01-01T13"):
        raise InputRangeError("start_date", "1979-01-01 to yesterday")
    if end > pd.Timestamp.now() - pd.Timedelta("1D"):
        raise InputRangeError("end_date", "1979-01-01 to yesterday")
    if end <= start:
        raise InputRangeError("end_date", "after start_date")

    dates = pd.date_range(start, end, freq="10000D").tolist()
    dates = dates + [end] if dates[-1] < end else dates

    if source == "grib":
        source_tag = "NLDAS:NLDAS_FORA0125_H.002"
        nldas_vars = NLDAS_VARS_GRIB
    elif source == "netcdf":
        source_tag = "NLDAS_FORA0125_H_v2.0"
        nldas_vars = NLDAS_VARS_NETCDF
    else:
        raise InputValueError(f"Source: {source} is invalid", ["grib", "netcdf"])

    if variables is None:
        clm_vars = [f"{source_tag}:{d['nldas_name']}" for d in nldas_vars.values()]
    else:
        clm_vars = [variables] if isinstance(variables, str) else list(variables)
        clm_vars = clm_vars + ["temp"] if snow and "temp" not in clm_vars else clm_vars
        if any(v not in nldas_vars for v in clm_vars):
            raise InputValueError("variables", list(nldas_vars))
        clm_vars = [f"{source_tag}:{nldas_vars[v]['nldas_name']}" for v in clm_vars]

    return dates, clm_vars, nldas_vars

In [12]:
dates, clm_vars, nldas_vars = check_inputs(START, END, VAR, False, source=SOURCE_GRIB)

In [13]:
clm_vars

['NLDAS:NLDAS_FORA0125_H.002:APCPsfc', 'NLDAS:NLDAS_FORA0125_H.002:PEVAPsfc']